# Decision Tree for HR Attrition Prediction

This notebook demonstrates how to use a Decision Tree Classifier in Python to predict employee attrition using HR data. It is designed for learners with basic Python skills and includes step-by-step code with explanations.

## 🎯 Learning Objectives
By the end of this notebook, you will be able to:
- Understand the intuition and structure behind Decision Trees
- Train a Decision Tree Classifier using `scikit-learn`
- Interpret model output to identify key attrition drivers
- Connect insights to real-world HR decisions

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.metrics import accuracy_score, confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt

In [ ]:
# Make sure this CSV is uploaded in your environment
df = pd.read_csv("WA_Fn-UseC_-HR-Employee-Attrition.csv")
df.head()

In [ ]:
df = df[["Attrition", "Age", "JobSatisfaction", "MonthlyIncome", "DistanceFromHome", "OverTime"]]
df['Attrition'] = df['Attrition'].map({'Yes': 1, 'No': 0})
df['OverTime'] = df['OverTime'].map({'Yes': 1, 'No': 0})
df.head()

In [ ]:
X = df.drop("Attrition", axis=1)
y = df["Attrition"]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
model = DecisionTreeClassifier(max_depth=3, random_state=42)
model.fit(X_train, y_train)

In [ ]:
y_pred = model.predict(X_test)
print("Accuracy Score:", accuracy_score(y_test, y_pred))

In [ ]:
plt.figure(figsize=(15, 8))
plot_tree(model, feature_names=X.columns, class_names=["Stay", "Leave"], filled=True)
plt.title("Decision Tree to Predict Employee Attrition")
plt.show()

In [ ]:
ConfusionMatrixDisplay.from_estimator(model, X_test, y_test, display_labels=["Stay", "Leave"])
plt.title("Confusion Matrix")
plt.show()

In [ ]:
importance = pd.Series(model.feature_importances_, index=X.columns).sort_values(ascending=False)
print("Feature Importances:\n", importance)

## 🧠 Reflection Questions
- Which features are most influential in predicting attrition?
- How would you act on this insight as an HR leader?
- Would you use this model to automate decisions or guide conversations?
